In [ ]:
## Import libraries
import numpy as np
import pandas as pd
import time
import json
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Flatten, concatenate, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD
import tensorflow.keras.backend as K
import gc
from hyperopt import fmin, tpe, hp, space_eval, Trials, STATUS_OK #, rand
from tensorflow.keras.utils import plot_model

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from Utils import WeightedError, Specificity, evaluate_model_skl, store_results, visualize_boxplots, visualize_boxplot_onemodel, compare_models

In [ ]:
pd.set_option('display.width', 1000)

In [ ]:
N = 10  # experiment repetitions
seed = 42
epochs = 30
batch_size = 8

k = 5  # k for k-fold cross-validation in hyperparameter tuning
seed_tuning = 13
max_evals = 30

# Load data

In [ ]:
from PIL import Image

## Frontal images
front_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/Front',f))) for f in os.listdir('Images/Healthy/Front')])
front_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/Front',f))) for f in os.listdir('Images/Sick/Front')])
front = np.concatenate((front_images_h, front_images_s))

## Left lateral (L90) images
L90_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/L90',f))) for f in os.listdir('Images/Healthy/L90')])
L90_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/L90',f))) for f in os.listdir('Images/Sick/L90')])
L90 = np.concatenate((L90_images_h, L90_images_s))

## Right lateral (R90) images
R90_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/R90',f))) for f in os.listdir('Images/Healthy/R90')])
R90_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/R90',f))) for f in os.listdir('Images/Sick/R90')])
R90 = np.concatenate((R90_images_h, R90_images_s))

In [ ]:
## Shape of thermograms
_,h,w = front.shape

In [ ]:
## Generate labels
labels_h = [0]*len(front_images_h)
labels_s = [1]*len(front_images_s)
labels = np.concatenate((labels_h, labels_s))

# Pre-processing

In [ ]:
## Min-max normalization
M = np.concatenate((front, L90, R90)).max()
m = np.concatenate((front, L90, R90))).min()

front = ((front - m) / (M - m)).astype('float32')
L90 = ((L90 - m) / (M - m)).astype('float32')
R90 = ((R90 - m) / (M - m)).astype('float32')

# Function definitions

In [ ]:
from tensorflow.keras.callbacks import Callback
class NaNCheckCallback(Callback):
    def on_batch_end(self, batch, logs=None):
        if np.isnan(logs.get('loss')):
            print(f'NaN value encountered at batch {batch}')
            self.model.stop_training = True

In [ ]:
## Function to get the data for hyperparameter tuning
def data():    
    ## Frontal images
    front_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/Front',f))) for f in os.listdir('Images/Healthy/Front')])
    front_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/Front',f))) for f in os.listdir('Images/Sick/Front')])
    front = np.concatenate((front_images_h, front_images_s))

    ## Left lateral (L90) images
    L90_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/L90',f))) for f in os.listdir('Images/Healthy/L90')])
    L90_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/L90',f))) for f in os.listdir('Images/Sick/L90')])
    L90 = np.concatenate((L90_images_h, L90_images_s))

    ## Right lateral (R90) images
    R90_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/R90',f))) for f in os.listdir('Images/Healthy/R90')])
    R90_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/R90',f))) for f in os.listdir('Images/Sick/R90')])
    R90 = np.concatenate((R90_images_h, R90_images_s))
    
    ## Labels
    labels_h = [0]*len(front_images_h)
    labels_s = [1]*len(front_images_s)
    labels = np.concatenate((labels_h, labels_s))

    ## Split the dataset into crossval and test sets
    seed_tuning = 13  # Seed for hyperparameter tuning
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=int(round(0.15*len(labels))), random_state = seed_tuning)
    crossval_index, test_index = [[crossval_index, test_index] for crossval_index, test_index in splitter.split(front, labels)][0]
    
    front_crossval = front[crossval_index]
    L90_crossval = L90[crossval_index]
    R90_crossval = R90[crossval_index]
    labels_crossval = labels[crossval_index]

    #front_test = front[test_index]
    #L90_test = L90[test_index]
    #R90_tets = R90[test_index]
    #labels_test = labels[test_index]
    
    ## add the channels dimension
    front_crossval = np.expand_dims(front_crossval,-1)
    L90_crossval = np.expand_dims(L90_crossval,-1)
    R90_crossval = np.expand_dims(R90_crossval,-1)

    #front_test = np.expand_dims(front_test,-1)
    #L90_test = np.expand_dims(L90_test,-1)
    #R90_tets = np.expand_dims(R90_tets,-1)

    return front_crossval, L90_crossval, R90_crossval, labels_crossval

# Custom CNN

## Tune hyperparameters (model architecture and learning rate)

In [ ]:
## Function to check tensor size to see if another downsampling layer can be applied
def can_apply_layer(x, min_height, min_width):
    """Check if the current layer can be applied based on tensor dimensions."""
    shape = K.int_shape(x)
    if shape[1] is None or shape[2] is None:
        return False  # Dynamic shape, cannot evaluate
    return shape[1] >= min_height and shape[2] >= min_width

### Single-input CNN

In [ ]:
## Single-input model
def create_singleInput_model(params, a=480, b=640):
            
    # Input
    stacked_input = Input(shape=(a, b, 3))
    
    x = Conv2D(32, (3, 3), activation='relu')(stacked_input)
    x = Conv2D(params['conv1_filters'], kernel_size=params['conv1_kernel'], strides=params['conv1_strides'], activation='relu')(x)
    x = MaxPooling2D(pool_size=params['pool1_size'])(x)
    
    x = Conv2D(64, kernel_size=params['conv2_kernel'], strides=params['conv2_strides'], activation='relu')(x)
    if params['add_conv_3']:
        x = Conv2D(64, kernel_size=params['conv3_kernel'], activation='relu')(x)
    if params['add_conv_4']:
        x = Conv2D(64, kernel_size=params['conv4_kernel'], activation='relu')(x)
        x = MaxPooling2D((2, 2))(x)
        
    if can_apply_layer(x, 5, 5):
        if params['add_conv_5']:
            x = Conv2D(params['conv5_filters'], kernel_size=params['conv5_kernel'], activation='relu')(x)
    if can_apply_layer(x, 5, 5):
        if params['add_conv_6']:
            x = Conv2D(params['conv6_filters'], kernel_size=params['conv6_kernel'], activation='relu')(x)
    if can_apply_layer(x, 5, 5):
        if params['add_conv_7']:
            x = Conv2D(params['conv7_filters'], kernel_size=params['conv7_kernel'], activation='relu')(x)
            if can_apply_layer(x, 2, 2):
                x = MaxPooling2D((2, 2))(x)
                
    if can_apply_layer(x, 5, 5):
        if params['add_conv_8']:
            x = Conv2D(128, kernel_size=params['conv8_kernel'], activation='relu')(x)
    if can_apply_layer(x, 5, 5):
        if params['add_conv_9']:
            x = Conv2D(128, kernel_size=params['conv9_kernel'], activation='relu')(x)
    if can_apply_layer(x, 5, 5):
        if params['add_conv_10']:
            x = Conv2D(128, kernel_size=params['conv10_kernel'], activation='relu')(x)
            if can_apply_layer(x, 2, 2):
                x = MaxPooling2D((2, 2))(x)
                
    if can_apply_layer(x, 5, 5):
        if params['add_conv_11']:
            x = Conv2D(128, kernel_size=params['conv11_kernel'], activation='relu')(x)
    if can_apply_layer(x, 5, 5):
        if params['add_conv_12']:
            x = Conv2D(128, kernel_size=params['conv12_kernel'], activation='relu')(x)
            if can_apply_layer(x, 2, 2):
                x = MaxPooling2D((2, 2))(x)
        
    if can_apply_layer(x, 10, 10):
        if params['add_conv_13']:
            x = Conv2D(128, kernel_size=params['conv13_kernel'], activation='relu')(x)
            x = MaxPooling2D((2, 2))(x)
        
    x = Flatten()(x)
    
    x = Dense(params['dense1_units'], activation='relu')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x) 
    
    if params['add_dense_2']:
        x = Dense(params['dense2_units'], use_bias=False)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.5)(x)   #new

    x = Dense(params['dense3_units'], activation='relu')(x)
    if params['add_dense_4']:
        x = Dense(params['dense4_units'], use_bias=False)(x)
    X = Dense(1, activation='sigmoid')(x)   
    
    ## create the model
    return Model(stacked_input, X)


# Objective function to minimize
def objective_singleInput(params):

    # Load the data
    front, L90, R90, labels = data()
    _,h,w,_ = front.shape

    # Initialize cross-validation
    epochs = 30
    batch_size = 8
    lr = params['lr']
    folds = 5
    seed = 13
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    # Store results from each fold
    fold_metrics = []

    # Initialize the model
    model = create_singleInput_model(params, h, w)

    # Save the initial weights of the model
    initial_weights = model.get_weights()

    for i, (train_idx, val_idx) in enumerate(skf.split(front, labels)):
        
        print(f"Fold {i+1}/{folds}")
        
        # Split the data into training and validation sets
        train_front, val_front = front[train_idx], front[val_idx]
        train_L90, val_L90 = L90[train_idx], L90[val_idx]
        train_R90, val_R90 = R90[train_idx], R90[val_idx]
        train_labels, val_labels = labels[train_idx], labels[val_idx]

        rate_train = sum(train_labels==0)/sum(train_labels)

        # Stack views along the channel axis
        train_images = np.concatenate((train_front, train_L90, train_R90), axis=-1)
        val_images = np.concatenate((val_front, val_L90, val_R90), axis=-1)

        # Reset the model to initial weights
        model.set_weights(initial_weights)

        # Train the model
        optimizer = SGD(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

        history  = model.fit(
            train_images, train_labels,
            class_weight={0: 1, 1: rate_train},
            validation_data=(val_images, val_labels),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[NaNCheckCallback()],
            verbose=0  # Suppress training output
        )

        # Record the best validation AUC for this fold
        best_auc = max(history.history['val_AUC'])
        fold_metrics.append(best_auc)

        print()

    # Clear TensorFlow session to release GPU memory
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    # Return the negative mean AUC across all folds as the loss
    print(f'Max test ROC AUC in each fold: {fold_metrics}')
    mean_auc = np.mean(fold_metrics)
    return -mean_auc  # Maximize AUC by minimizing its negative


## Hyperparameter search space
space = {
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-2)),

    'conv1_filters': hp.choice('conv1_filters', [32, 64]),
    'conv1_kernel': hp.choice('conv1_kernel', [(3, 3), (5, 5)]),
    'conv1_strides': hp.choice('conv1_strides', [1, 2]),
    'pool1_size': hp.choice('pool1_size', [2, 3]),
    'conv2_kernel': hp.choice('conv2_kernel', [(3, 3), (5, 5)]),
    'conv2_strides': hp.choice('conv2_strides', [1, 2]),
    'add_conv_3' : hp.choice('add_conv_3', [True, False]),
    'conv3_kernel': hp.choice('conv3_kernel', [(3, 3), (5, 5)]),
    'add_conv_4' : hp.choice('add_conv_4', [True, False]),
    'conv4_kernel': hp.choice('conv4_kernel', [(3, 3), (5, 5)]),
    'add_conv_5' : hp.choice('add_conv_5', [True, False]),
    'conv5_filters': hp.choice('conv5_filters', [64, 128]),
    'conv5_kernel': hp.choice('conv5_kernel', [(3, 3), (5, 5)]),
    'add_conv_6' : hp.choice('add_conv_6', [True, False]),
    'conv6_filters': hp.choice('conv6_filters', [64, 128]),
    'conv6_kernel': hp.choice('conv6_kernel', [(3, 3), (5, 5)]),
    'add_conv_7' : hp.choice('add_conv_7', [True, False]),
    'conv7_filters': hp.choice('conv7_filters', [64, 128]),
    'conv7_kernel': hp.choice('conv7_kernel', [(3, 3), (5, 5)]),
    'add_conv_8' : hp.choice('add_conv_8', [True, False]),
    'conv8_kernel': hp.choice('conv8_kernel', [(3, 3), (5, 5)]),
    'add_conv_9' : hp.choice('add_conv_9', [True, False]),
    'conv9_kernel': hp.choice('conv9_kernel', [(3, 3), (5, 5)]),
    'add_conv_10' : hp.choice('add_conv_10', [True, False]),
    'conv10_kernel': hp.choice('conv10_kernel', [(3, 3), (5, 5)]),
    'add_conv_11' : hp.choice('add_conv_11', [True, False]),
    'conv11_kernel': hp.choice('conv11_kernel', [(3, 3), (5, 5)]),
    'add_conv_12' : hp.choice('add_conv_12', [True, False]),
    'conv12_kernel': hp.choice('conv12_kernel', [(3, 3), (5, 5)]),
    'add_conv_13' : hp.choice('add_conv_13', [True, False]),
    'conv13_kernel': hp.choice('conv13_kernel', [(3, 3), (5, 5)]),
    
    'dense1_units': hp.choice('dense1_units', [256, 512]),
    'add_dense_2' : hp.choice('add_dense_2', [True, False]),
    'dense2_units': hp.choice('dense2_units', [64, 128]),
    'dense3_units': hp.choice('dense3_units', [32, 64]),
    'add_dense_4' : hp.choice('add_dense_4', [True, False]),
    'dense4_units': hp.choice('dense4_units', [32, 64])
}

In [ ]:
# Create trials object to store optimization results
trials = Trials()

# Run optimization
ti = time.time()
best_singleInput = fmin(
    fn=objective_singleInput,
    space=space,
    algo=tpe.suggest,
    max_evals=max_evals,
    trials=trials
)
tuningTime = time.time() - ti

hours, remainder = divmod(tuningTime, 3600)
minutes, seconds = divmod(remainder, 60)
print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

print("Best hyperparameters:", space_eval(space, best_singleInput))

# Save the results
json_compatible_params = {
    key: (list(value) if isinstance(value, tuple) else value)
    for key, value in space_eval(space, best_singleInput).items()
}
with open('Image_classifiers/Parameters_singleInputCNN.json', "w") as f:
    json.dump(json_compatible_params, f)

In [ ]:
model = create_singleInput_model(space_eval(space, best_singleInput), h, w)
plot_model(model, to_file='Image_classifiers/Models/singleInputCNN.png', show_shapes=True, show_layer_names=True)

### Multi-input CNN

In [ ]:
## Multi-input model
def create_multiInput_model(params, a=480, b=640):

    ## Front
    front_input = Input(shape=(a, b, 1))
    front = Conv2D(32, (3, 3), activation='relu')(front_input)
    front = Conv2D(64, kernel_size=params['front_conv1_kernel'], strides=params['front_conv1_strides'], activation='relu')(front)  # strides=2
    front = MaxPooling2D((2, 2))(front)
    front = Conv2D(params['front_conv2_filters'], kernel_size=params['front_conv2_kernel'], activation='relu')(front)  #256, 128
    front = Conv2D(params['front_conv3_filters'], kernel_size=params['front_conv3_kernel'], activation='relu')(front)  #64
    front = MaxPooling2D((2, 2))(front)
    front = Flatten()(front) 
    front = Dense(params['front_dense1_units'], activation='relu')(front)  #256, 512
    if params['front_add_dense2']:
        front = Dropout(0.5)(front)
        front = Dense(params['front_dense2_units'], activation='relu')(front)  #128, 256 
    
    ## L90
    L90_input = Input(shape=(a, b, 1))
    L90 = Conv2D(32, (3, 3), activation='relu')(L90_input)
    L90 = Conv2D(params['L90_conv1_filters'], kernel_size=params['L90_conv1_kernel'], activation='relu')(L90)  #64
    L90 = MaxPooling2D((2, 2))(L90)
    L90 = Conv2D(params['L90_conv2_filters'], kernel_size=params['L90_conv2_kernel'], strides=2, activation='relu')(L90)  #128
    #L90 = Conv2D(64, (5, 5), activation='relu')(L90) 
    if params['L90_add_conv3']:
        L90 = Conv2D(params['L90_conv3_filters'], kernel_size=params['L90_conv3_kernel'], activation='relu')(L90) 
    L90 = Conv2D(params['L90_conv4_filters'], kernel_size=params['L90_conv4_kernel'], activation='relu')(L90) # 64, (5, 5)
    L90 = MaxPooling2D((2, 2))(L90)
    L90 = Flatten()(L90)
    L90 = Dense(256, activation='relu')(L90)  
    #L90 = Dropout(0.5)(L90)
    #L90 = Dense(128, activation='relu')(L90)
    
    ## R90
    R90_input = Input(shape=(a, b, 1))
    R90 = Conv2D(32, (3, 3), activation='relu')(R90_input)
    R90 = Conv2D(params['R90_conv1_filters'], kernel_size=params['R90_conv1_kernel'], activation='relu')(R90)   
    R90 = MaxPooling2D((2, 2))(R90)
    R90 = Conv2D(params['R90_conv2_filters'], kernel_size=params['R90_conv2_kernel'], strides=2, activation='relu')(R90)  #128
    #R90 = Conv2D(64, (5, 5), activation='relu')(R90)  
    if params['R90_add_conv3']:
        R90 = Conv2D(params['R90_conv3_filters'], kernel_size=params['R90_conv3_kernel'], activation='relu')(R90) 
    R90 = Conv2D(params['R90_conv4_filters'], kernel_size=params['R90_conv4_kernel'], activation='relu')(R90)  # 64, (5, 5)
    R90 = MaxPooling2D((2, 2))(R90)
    R90 = Flatten()(R90)
    R90 = Dense(256, activation='relu')(R90)  
    #R90 = Dropout(0.5)(R90)
    #R90 = Dense(128, activation='relu')(R90)

    ## concatenate all the outputs of each stalk of the net.
    concatenated = concatenate([front, L90, R90], axis=-1)
    x = Dense(params['dense1_units'], activation='relu')(concatenated)  ## antes 512, 256
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x) 

    if params['add_dense2']:
        x = Dense(params['dense2_units'], use_bias=False)(x)  ## 64
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        #x = Dense(64, activation='relu')(x)  ## 64, 128
        #x = Dense(32, activation='relu')(x)
        x = Dropout(0.5)(x)   #new

    x = Dense(params['dense3_units'], activation='relu')(x)   # <-- batchnorm en esta capa funciona mal
    X = Dense(1, activation='sigmoid')(x)   
            
    ## create the model
    return Model([front_input, L90_input, R90_input], X)


# Objective function to minimize
def objective_multiInput(params):

    # Load the data
    front, L90, R90, labels = data()
    _,h,w,_ = front.shape

    # Initialize cross-validation
    epochs = 30
    batch_size = 8
    lr = params['lr']
    folds = 5
    seed = 13
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    # Store results from each fold
    fold_metrics = []

    # Initialize the model
    model = create_multiInput_model(params, h, w)

    # Save the initial weights of the model
    initial_weights = model.get_weights()

    for i, (train_idx, val_idx) in enumerate(skf.split(front, labels)):
        
        print(f"Fold {i+1}/{folds}")
        
        # Split the data into training and validation sets
        train_front, val_front = front[train_idx], front[val_idx]
        train_L90, val_L90 = L90[train_idx], L90[val_idx]
        train_R90, val_R90 = R90[train_idx], R90[val_idx]
        train_labels, val_labels = labels[train_idx], labels[val_idx]

        rate_train = sum(train_labels==0)/sum(train_labels)

        # Stack views along the channel axis
        train_images = [train_front, train_L90, train_R90]
        val_images = [val_front, val_L90, val_R90]

        # Reset the model to initial weights
        model.set_weights(initial_weights)

        # Train the model
        optimizer = SGD(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

        history = model.fit(
            train_images, train_labels,
            class_weight={0: 1, 1: rate_train},
            validation_data=(val_images, val_labels),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[NaNCheckCallback()],
            verbose=0  # Suppress training output
        )

        # Record the best validation AUC for this fold
        best_auc = max(history.history['val_AUC'])
        fold_metrics.append(best_auc)

        print()

    # Clear TensorFlow session to release GPU memory
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    # Return the negative mean AUC across all folds as the loss
    print(f'Max test ROC AUC in each fold: {fold_metrics}')
    mean_auc = np.mean(fold_metrics)
    return -mean_auc  # Maximize AUC by minimizing its negative


## Hyperparameter search space
space = {
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-2)),
    
    'front_conv1_kernel': hp.choice('front_conv1_kernel', [(3, 3), (5, 5)]),
    'front_conv1_strides': hp.choice('front_conv1_strides', [1, 2]),
    'front_conv2_filters': hp.choice('front_conv2_filters', [64, 128, 256]),
    'front_conv2_kernel': hp.choice('front_conv2_kernel', [(3, 3), (5, 5)]),
    'front_conv3_filters': hp.choice('front_conv3_filters', [64, 128]),
    'front_conv3_kernel': hp.choice('front_conv3_kernel', [(3, 3), (5, 5)]),
    'front_dense1_units': hp.choice('front_dense1_units', [256, 512]),
    'front_add_dense2' : hp.choice('front_add_dense2', [True, False]),
    'front_dense2_units': hp.choice('front_dense2_units', [128, 256]),

    'L90_conv1_filters': hp.choice('L90_conv1_filters', [32, 64]),
    'L90_conv1_kernel': hp.choice('L90_conv1_kernel', [(3, 3), (5, 5)]),
    'L90_conv2_filters': hp.choice('L90_conv2_filters', [64, 128]),
    'L90_conv2_kernel': hp.choice('L90_conv2_kernel', [(3, 3), (5, 5)]),
    'L90_add_conv3' : hp.choice('L90_add_conv3', [True, False]),
    'L90_conv3_filters': hp.choice('L90_conv3_filters', [64, 128]),
    'L90_conv3_kernel': hp.choice('L90_conv3_kernel', [(3, 3), (5, 5)]),
    'L90_conv4_filters': hp.choice('L90_conv4_filters', [64, 128]),
    'L90_conv4_kernel': hp.choice('L90_conv4_kernel', [(3, 3), (5, 5)]),

    'R90_conv1_filters': hp.choice('R90_conv1_filters', [32, 64]),
    'R90_conv1_kernel': hp.choice('R90_conv1_kernel', [(3, 3), (5, 5)]),
    'R90_conv2_filters': hp.choice('R90_conv2_filters', [64, 128]),
    'R90_conv2_kernel': hp.choice('R90_conv2_kernel', [(3, 3), (5, 5)]),
    'R90_add_conv3' : hp.choice('R90_add_conv3', [True, False]),
    'R90_conv3_filters': hp.choice('R90_conv3_filters', [64, 128]),
    'R90_conv3_kernel': hp.choice('R90_conv3_kernel', [(3, 3), (5, 5)]),
    'R90_conv4_filters': hp.choice('R90_conv4_filters', [64, 128]),
    'R90_conv4_kernel': hp.choice('R90_conv4_kernel', [(3, 3), (5, 5)]),

    'dense1_units': hp.choice('dense1_units', [256, 512]),
    'add_dense2' : hp.choice('add_dense2', [True, False]),
    'dense2_units': hp.choice('dense2_units', [64, 128]),
    'dense3_units': hp.choice('dense3_units', [32, 64])
}

In [ ]:
# Create trials object to store optimization results
trials = Trials()

# Run optimization
ti = time.time()
best_multiInput = fmin(
    fn=objective_multiInput,
    space=space,
    algo=tpe.suggest,
    max_evals=max_evals,
    trials=trials
)
tuningTime = time.time() - ti

hours, remainder = divmod(tuningTime, 3600)
minutes, seconds = divmod(remainder, 60)
print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

print("Best hyperparameters:", space_eval(space, best_multiInput))

# Save the results
json_compatible_params = {
    key: (list(value) if isinstance(value, tuple) else value)
    for key, value in space_eval(space, best_multiInput).items()
}
with open('Image_classifiers/Parameters_multiInputCNN.json', "w") as f:
    json.dump(json_compatible_params, f)

In [ ]:
model = create_multiInput_model(space_eval(space, best_multiInput), h, w)
plot_model(model, to_file='Image_classifiers/Models/multiInputCNN.png', show_shapes=True, show_layer_names=True)

## Train N times

In [ ]:
trials_results = []

### Single-input CNN

In [ ]:
# Read tuned hyperparameters
with open("Image_classifiers/Parameters_singleInputCNN.json", "r") as f:
    params = json.load(f)
    
lr = params['lr']
params.pop('lr')

# Convert lists back to tuples if needed
params = {
    key: (tuple(value) if isinstance(value, list) else value)
    for key, value in params.items()
}

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

for trial, (train_index, test_index) in enumerate(splitter.split(front, labels)):

    print(f'Trial {trial + 1}'), print()

    # Split the dataset
    train_front = front[train_index]
    train_L90 = L90[train_index]
    train_R90 = R90[train_index]
    train_labels = labels[train_index]
    
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    # Stack the three views
    train_images = np.stack((train_front, train_L90, train_R90), axis=-1)
    test_images = np.stack((test_front, test_L90, test_R90), axis=-1)

    # Compute the class weight difference
    rate_train = sum(train_labels == 0) / sum(train_labels)

    # Initialize and compile the model
    model = create_singleInput_model(params, h, w)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision', 'Recall', Specificity(), WeightedError(rate=20)], 
                  optimizer=SGD(learning_rate=lr, clipvalue=1.0))
    
    # Train the model
    ti = time.time()
    history = model.fit(train_images, train_labels, class_weight = {0: 1, 1: rate_train}, 
                        validation_data = (test_images, test_labels),
                        batch_size=batch_size, epochs=epochs, verbose=0, callbacks=[NaNCheckCallback()]) 
    trainTime = time.time() - ti

    hours, remainder = divmod(trainTime, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Training took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    # Save learning curves
    plt.figure(figsize=(30, 10))
    plt.subplot(1, 3, 1)
    plt.plot(range(epochs), history.history['loss'], label='Train Loss')
    plt.plot(range(epochs), history.history['val_loss'], label='Test Loss')
    plt.legend(loc='upper right',fontsize=10)
    plt.title('Train and Test Loss',fontsize=12)
    
    plt.subplot(1, 3, 2)
    plt.plot(range(epochs), history.history['accuracy'], label='Training Accuracy')
    plt.plot(range(epochs), history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend(loc='lower right',fontsize=10)
    plt.title('Train and Test Accuracy',fontsize=12)
    
    plt.subplot(1, 3, 3)
    plt.plot(range(epochs), history.history['AUC'], label='Train AUC')
    plt.plot(range(epochs), history.history['val_AUC'], label='Test AUC')
    plt.legend(loc='lower right',fontsize=10)
    plt.title('Train and Test ROC AUC',fontsize=12)
    
    plt.savefig('Image_classifiers/Learning_curves/SingleInputCNN_'+str(trial+1)+'.png'), plt.show()
    plt.show()

    # Save the model
    model.save('Image_classifiers/Models/SingleInputCNN_'+str(trial+1)+'.h5')

    # Print the number of parameters in the model
    total_params = model.count_params()  # total_params = np.sum([np.prod(v.shape.as_list()) for v in model.variables])
    trainable_params = np.sum([np.prod(v.shape.as_list()) for v in model.trainable_variables])
    print(f'Classifier has {total_params} total parameters, {trainable_params} of which are trainable.'), print()
    
    # Predict
    train_preds = model.predict(train_images)
    np.save('Image_classifiers/Predictions/SingleInputCNN_train_'+str(trial+1)+'.npy',train_preds)
    test_preds = model.predict(test_images)
    np.save('Image_classifiers/Predictions/SingleInputCNN_test_'+str(trial+1)+'.npy',test_preds)

    # Evaluate the model
    results_train = evaluate_model_skl(train_preds, train_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    results_test = evaluate_model_skl(test_preds, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append({**{'classifier':'singleInputCNN'}, **{'trial':trial+1}, 
                            **store_results(trainable_params, trainTime, results_train, results_test)})

    # Clear TensorFlow session to release GPU memory
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    print(), print(100*'#'), print()

pd.DataFrame(trials_results).to_csv('Image_classifiers/Results_'+str(N)+'trials.csv')

### Multi-input CNN

In [ ]:
# Read tuned hyperparameters
with open("Image_classifiers/Parameters_multiInputCNN.json", "r") as f:
    params = json.load(f)
    
lr = params['lr']
params.pop('lr')

# Convert lists back to tuples if needed
params = {
    key: (tuple(value) if isinstance(value, list) else value)
    for key, value in params.items()
}

In [ ]:
trials_results = pd.read_csv('Image_classifiers/Results_'+str(N)+'trials.csv', index_col = 0).to_dict(orient='records')

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

for trial, (train_index, test_index) in enumerate(splitter.split(front, labels)):

    print(f'Trial {trial + 1}'), print()

    # Split the dataset
    train_front = front[train_index]
    train_L90 = L90[train_index]
    train_R90 = R90[train_index]
    train_labels = labels[train_index]
    
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    # Add the channel dimension
    train_front = np.expand_dims(train_front, -1)
    train_L90 = np.expand_dims(train_L90, -1)
    train_R90 = np.expand_dims(train_R90, -1)

    test_front = np.expand_dims(test_front, -1)
    test_L90 = np.expand_dims(test_L90, -1)
    test_R90 = np.expand_dims(test_R90, -1)

    # Compute the class weight difference
    rate_train = sum(train_labels == 0) / sum(train_labels)

    # Initialize and compile the model
    model = create_multiInput_model(params, h, w)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision', 'Recall', Specificity(), WeightedError(rate=20)], 
                  optimizer=SGD(learning_rate=lr, clipvalue=1.0))
    
    # Train the model
    ti = time.time()
    history = model.fit([train_front, train_L90, train_R90], train_labels, class_weight = {0: 1, 1: rate_train}, 
                        validation_data = ([test_front, test_L90, test_R90], test_labels)
                        batch_size=batch_size, epochs=epochs, verbose=0, callbacks=[NaNCheckCallback()]) 
    trainTime = time.time() - ti

    hours, remainder = divmod(trainTime, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Training took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    # Save learning curves
    plt.figure(figsize=(30, 10))
    plt.subplot(1, 3, 1)
    plt.plot(range(epochs), history.history['loss'], label='Train Loss')
    plt.plot(range(epochs), history.history['val_loss'], label='Test Loss')
    plt.legend(loc='upper right',fontsize=10)
    plt.title('Train and Test Loss',fontsize=12)
    
    plt.subplot(1, 3, 2)
    plt.plot(range(epochs), history.history['accuracy'], label='Training Accuracy')
    plt.plot(range(epochs), history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend(loc='lower right',fontsize=10)
    plt.title('Train and Test Accuracy',fontsize=12)
    
    plt.subplot(1, 3, 3)
    plt.plot(range(epochs), history.history['AUC'], label='Train AUC')
    plt.plot(range(epochs), history.history['val_AUC'], label='Test AUC')
    plt.legend(loc='lower right',fontsize=10)
    plt.title('Train and Test ROC AUC',fontsize=12)
    
    plt.savefig('Image_classifiers/Learning_curves/MultiInputCNN_'+str(trial+1)+'.png'), plt.show()
    plt.show()

    # Save the model
    model.save('Image_classifiers/Models/MultiInputCNN_'+str(trial+1)+'.h5')

    # Print the number of parameters in the model
    total_params = model.count_params()  # total_params = np.sum([np.prod(v.shape.as_list()) for v in model.variables])
    trainable_params = np.sum([np.prod(v.shape.as_list()) for v in model.trainable_variables])
    print(f'Classifier has {total_params} total parameters, {trainable_params} of which are trainable.'), print()
    
    # Predict
    train_preds = model.predict([train_front, train_L90, train_R90])
    np.save('Image_classifiers/Predictions/MultiInputCNN_train_'+str(trial+1)+'.npy',train_preds)
    test_preds = model.predict([test_front, test_L90, test_R90])
    np.save('Image_classifiers/Predictions/MultiInputCNN_test_'+str(trial+1)+'.npy',test_preds)

    # Evaluate the model
    results_train = evaluate_model_skl(train_preds, train_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    results_test = evaluate_model_skl(test_preds, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append({**{'classifier':'multiInputCNN'}, **{'trial':trial+1}, 
                            **store_results(trainable_params, trainTime, results_train, results_test)})

    # Clear TensorFlow session to release GPU memory
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    print(), print(100*'#'), print()

pd.DataFrame(trials_results).to_csv('Image_classifiers/Results_'+str(N)+'trials.csv')

# Pre-trained CNNs

In [ ]:
## Import pre-trained CNNs
from tensorflow.keras.applications import DenseNet121, DenseNet169, DenseNet201, VGG16, VGG19, ResNet50, ResNet101, ResNet152, InceptionV3, MobileNet, MobileNetV3Small

In [ ]:
# Dictionary of models
pretrained_dict = {
    'VGG16': VGG16,
    'VGG19': VGG19,
    'ResNet50': ResNet50,
    'ResNet101': ResNet101,
    'ResNet152': ResNet152,
    'DenseNet121': DenseNet121,
    'DenseNet169': DenseNet169,
    'DenseNet201': DenseNet201,
    'InceptionV3': InceptionV3,
    'MobileNet': MobileNet,
    'MobileNetV3Small': MobileNetV3Small
}

In [ ]:
# Percentage of top layers to unfreeze during second step of fine-tuning
perc_unfreeze = 0.2

# Number of epochs for the first step of the fine-tuning process
epochs_freeze = epochs//2

## Tune learning rate

In [ ]:
## Objective function to minimize
def objective(params, baseModel_fn):

    # Load the data
    front, L90, R90, labels = data()
    _,h,w,_ = front.shape

    # Initialize cross-validation
    folds = 5
    seed = 13
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    # Store results from each fold
    fold_metrics = []

    # Load the base model
    baseModel = baseModel_fn(weights='imagenet', include_top=False, input_shape=(h, w, 3))

    # Initially freeze all layers of the base model
    for layer in baseModel.layers:
        layer.trainable = False

    # Build the full model
    model = Sequential()
    model.add(baseModel)
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Save the initial weights of the model
    initial_weights = model.get_weights()

    epochs = 30
    epochs_freeze = epochs//2
    batch_size = 8
    perc_unfreeze = 0.2
    lr_freeze = params['lr_freeze']
    lr_unfreeze = params['lr_unfreeze']
    
    for i, (train_idx, val_idx) in enumerate(skf.split(front, labels)):
        
        print(f"Fold {i+1}/{folds}")
        
        # Split the data into training and validation sets
        train_front, val_front = front[train_idx], front[val_idx]
        train_L90, val_L90 = L90[train_idx], L90[val_idx]
        train_R90, val_R90 = R90[train_idx], R90[val_idx]
        train_labels, val_labels = labels[train_idx], labels[val_idx]

        rate_train = sum(train_labels==0)/sum(train_labels)

        # Stack views along the channel axis
        train_images = np.concatenate((train_front, train_L90, train_R90), axis=-1)
        val_images = np.concatenate((val_front, val_L90, val_R90), axis=-1)

        # Reset the model to initial weights
        model.set_weights(initial_weights)

        # Step 1: Train only the fully connected layers
        optimizer = SGD(learning_rate=lr_freeze)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

        model.fit(
            train_images, train_labels,
            class_weight={0: 1, 1: rate_train},
            epochs=epochs_freeze,
            batch_size=batch_size,
            callbacks=[NaNCheckCallback()],
            verbose=0  # Suppress training output
        )

        test_AUC_1 = model.evaluate(val_images, val_labels, verbose=0)[1]
        print(f'Test ROC AUC after step 1 of fine-tuning: {test_AUC_1}')


        # Step 2: Unfreeze part of the base model
        total_layers = len(baseModel.layers)
        unfreeze_layers = int(total_layers * perc_unfreeze)

        for layer in baseModel.layers[total_layers - unfreeze_layers:]:
            layer.trainable = True

        # Recompile the model with a reduced learning rate
        optimizer = SGD(learning_rate=lr_unfreeze)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

        # Train the model for fine-tuning
        history = model.fit(
            train_images, train_labels,
            class_weight={0: 1, 1: rate_train},
            validation_data=(val_images, val_labels),
            epochs=epochs-epochs_freeze,
            batch_size=batch_size,
            callbacks=[NaNCheckCallback()],
            verbose=0
        )

        test_AUC_2 = model.evaluate(val_images, val_labels, verbose=0)[1]
        print(f'Test ROC AUC after step 2 of fine-tuning: {test_AUC_2}')

        # Record the best validation AUC for this fold
        best_auc = max(history.history['val_AUC'])
        fold_metrics.append(best_auc)

        print()

    # Clear TensorFlow session to release GPU memory
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    # Return the negative mean AUC across all folds as the loss
    mean_auc = np.mean(fold_metrics)
    return -mean_auc  # Maximize AUC by minimizing its negative


## Hyperparameter search space
space = {
    'lr_freeze': hp.loguniform('lr_freeze', np.log(1e-5), np.log(1e-2)),
    'lr_unfreeze': hp.loguniform('lr_unfreeze', np.log(1e-6), np.log(1e-3))
}

In [ ]:
learning_rates_pt = {}
for model_name, baseModel_fn in pretrained_dict.items():
    print(f'Base classifier: {model_name}')
    
    # Objective function for the specific pre-trained CNN
    objective_pt = lambda params: objective(params, baseModel_fn)

    # Create trials object to store optimization results
    trials = Trials()

    # Run optimization
    ti = time.time()
    best = fmin(
        fn=objective_pt,
        space=space,
        algo=tpe.suggest,
        max_evals=max_evals,
        trials = trials
    )
    tuningTime = time.time() - ti

    hours, remainder = divmod(tuningTime, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    print("Best hyperparameters:", best)

    learning_rates_pt[model_name] = {'lr_freeze' : best['lr_freeze'], 'lr_unfreeze' : best['lr_unfreeze']}

    # Plot the parameter tuning process
    lrs_freeze = np.asarray([trial['misc']['vals']['lr_freeze'][0] for trial in trials.trials])
    lrs_unfreeze = np.asarray([trial['misc']['vals']['lr_unfreeze'][0] for trial in trials.trials])
    losses = np.asarray([trial['result']['loss'] for trial in trials.trials])
    
    # Plot the results
    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(lrs_freeze, lrs_unfreeze, -losses, c=losses, cmap='viridis', label='Loss vs. Learning Rates')
    ax.set_xlabel('Learning Rate for Step 1')
    ax.set_ylabel('Learning Rate for Step 2')
    ax.set_zlabel('Mean ROC AUC')
    ax.set_title('Hyperparameter Tuning: ROC AUC vs. Learning Rate')
    plt.grid(True)
    plt.savefig('Image_classifiers/LearningRate_tuning/'+model_name+'.png'), plt.show()
    plt.show()
    
    print('-'*120), print()

pd.DataFrame(learning_rates_pt).to_csv('Image_classifiers/Parameters_preTrainedCNNs.csv')

## Train N times

In [ ]:
trials_results = pd.read_csv('Image_classifiers/Results_'+str(N)+'trials.csv', index_col = 0).to_dict(orient='records')
learning_rates_pt = pd.read_csv('Image_classifiers/Parameters_preTrainedCNNs.csv', index_col = 0)

In [ ]:
for model_name, baseModel_fn in pretrained_dict.items():
    if not model_name in learning_rates_pt.columns:
        continue
            
    print(f'Base classifier: {model_name}')

    splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

    # Get the tuned learning rates
    lr_freeze = learning_rates_pt[model_name]['lr_freeze']
    lr_unfreeze = learning_rates_pt[model_name]['lr_unfreeze']

    #best_test_auc = 0
    for trial, (train_index, test_index) in enumerate(splitter.split(front, labels)):
    
        print(f'Trial {trial + 1}'), print()
    
        ## Split the dataset
        train_front = front[train_index]
        train_L90 = L90[train_index]
        train_R90 = R90[train_index]
        train_labels = labels[train_index]
        
        test_front = front[test_index]
        test_L90 = L90[test_index]
        test_R90 = R90[test_index]
        test_labels = labels[test_index]
    
        # Stack the three views
        train_images = np.stack((train_front, train_L90, train_R90), axis=-1)
        test_images = np.stack((test_front, test_L90, test_R90), axis=-1)
    
        rate_train = sum(train_labels==0)/sum(train_labels)

        ## Build the model
        # Load the base model
        baseModel = baseModel_fn(weights='imagenet', include_top=False, input_shape=(h, w, 3))
    
        # Initially freeze all layers of the base model
        for layer in baseModel.layers:
            layer.trainable = False
    
        # Build the full model
        model = Sequential()
        model.add(baseModel)
        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(1, activation='sigmoid'))
        

        ## Train the model
        # Step 1: Train only the fully connected layers
        optimizer = SGD(learning_rate=lr_freeze, clipvalue=1.0)
        model.compile(loss='binary_crossentropy',
                      metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision', 'Recall', Specificity(), WeightedError(rate=20)], 
                      optimizer=optimizer)

        ti = time.time()
        history1 = model.fit(
            train_images, train_labels,
            class_weight={0: 1, 1: rate_train},
            validation_data=(test_images, test_labels),
            epochs=epochs_freeze,
            batch_size=batch_size,
            callbacks=[NaNCheckCallback()],
            verbose=0  # Suppress training output
        )

        test_AUC_1 = model.evaluate(test_images, test_labels, verbose=0)[2]
        print(f'Test ROC AUC after step 1 of fine-tuning: {test_AUC_1}')


        # Step 2: Unfreeze part of the base model
        total_layers = len(baseModel.layers)
        unfreeze_layers = int(total_layers * perc_unfreeze)

        for layer in baseModel.layers[total_layers - unfreeze_layers:]:
            layer.trainable = True

        # Recompile the model with a reduced learning rate
        optimizer = SGD(learning_rate=lr_unfreeze, clipvalue=1.0)
        model.compile(loss='binary_crossentropy',
                      metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision', 'Recall', Specificity(), WeightedError(rate=20)], 
                      optimizer=optimizer)

        # Train the model for fine-tuning
        history2 = model.fit(
            train_images, train_labels,
            class_weight={0: 1, 1: rate_train},
            validation_data=(test_images, test_labels),
            epochs=epochs-epochs_freeze,
            batch_size=batch_size,
            callbacks=[NaNCheckCallback()],
            verbose=0
        )

        test_AUC_2 = model.evaluate(test_images, test_labels, verbose=0)[2]
        print(f'Test ROC AUC after step 2 of fine-tuning: {test_AUC_2}')

        trainTime = time.time() - ti

        hours, remainder = divmod(trainTime, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f'Training took {hours} hours, {minutes} minutes, and {seconds} seconds.')

        # Save learning curves
        plt.figure(figsize=(30, 10))
        plt.subplot(1, 3, 1)
        plt.plot(range(epochs), history1.history['loss']+history2.history['loss'], label='Train Loss')
        plt.plot(range(epochs), history1.history['val_loss']+history2.history['val_loss'], label='Test Loss')
        plt.legend(loc='upper right',fontsize=10)
        plt.title('Train and Test Loss',fontsize=12)
        
        plt.subplot(1, 3, 2)
        plt.plot(range(epochs), history1.history['accuracy']+history2.history['accuracy'], label='Training Accuracy')
        plt.plot(range(epochs), history1.history['val_accuracy']+history2.history['val_accuracy'], label='Validation Accuracy')
        plt.legend(loc='lower right',fontsize=10)
        plt.title('Train and Test Accuracy',fontsize=12)
        
        plt.subplot(1, 3, 3)
        plt.plot(range(epochs), history1.history['AUC']+history2.history['AUC'], label='Train AUC')
        plt.plot(range(epochs), history1.history['val_AUC']+history2.history['val_AUC'], label='Test AUC')
        plt.legend(loc='lower right',fontsize=10)
        plt.title('Train and Test ROC AUC',fontsize=12)
        
        plt.savefig('Image_classifiers/Learning_curves/'+model_name+'_'+str(trial+1)+'.png'), plt.show()
        plt.show()

        
        # Save the model
        model.save('Image_classifiers/Models/'+model_name+'_'+str(trial+1)+'.h5')

        
        # Print the number of parameters in the model
        total_params = model.count_params()  # total_params = np.sum([np.prod(v.shape.as_list()) for v in model.variables])
        trainable_params = np.sum([np.prod(v.shape.as_list()) for v in model.trainable_variables])
        print(f'Classifier has {total_params} total parameters, {trainable_params} of which are trainable.'), print()
    
        # Predict
        train_preds = model.predict(train_images)
        np.save('Image_classifiers/Predictions/'+model_name+'_train_'+str(trial+1)+'.npy',train_preds)
        test_preds = model.predict(test_images)
        np.save('Image_classifiers/Predictions/'+model_name+'_test_'+str(trial+1)+'.npy',test_preds)

        # Evaluate the model
        results_train = evaluate_model_skl(train_preds, train_labels)
        print('TRAIN results:')
        for metric, value in results_train.items():
            print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
        print()

        results_test = evaluate_model_skl(test_preds, test_labels)
        print('TEST results:')
        for metric, value in results_test.items():
            print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
        print()

        ## Store results
        trials_results.append({**{'classifier':model_name}, **{'trial':trial+1}, 
                                **store_results(trainable_params, trainTime, results_train, results_test)})
    
        # Clear TensorFlow session to release GPU memory
        tf.keras.backend.clear_session()
        del model
        gc.collect()
    
        print(), print(100*'#'), print()

pd.DataFrame(trials_results).to_csv('Image_classifiers/Results_'+str(N)+'trials.csv')

# Compare models

In [ ]:
## Read results
trials_results = pd.read_csv('Image_classifiers/Results_'+str(N)+'trials.csv', index_col=0)
trials_results.fillna(1e-10, inplace=True)

In [ ]:
## Print statistics
models = trials_results.classifier.unique()
metrics = [c for c in trials_results.columns if 'test_' in c and c not in ['test_TP','test_FP','test_TN','test_FN']]
    
statistics = pd.DataFrame(index=models, columns=[item for sublist in [[metric+'_mean', metric+'_std'] for metric in metrics] for item in sublist])
    
for metric in metrics:
    mn, st = metric+'_mean', metric+'_std'
    for model in models:
        results = trials_results[trials_results['classifier']==model][metric].values
        statistics.at[model,mn] = results.mean()
        statistics.at[model,st] = results.std()

statistics

In [ ]:
for metric in [m for m in metrics if 'test' in m]:
    mn, st = metric+'_mean', metric+'_std'
    if 'Loss' in metric or 'WE' in metric:
        model_best = pd.to_numeric(statistics[metric+'_mean']).idxmin()
        print(f'Model with lowest {metric} is {model_best} with value {statistics.loc[model_best,mn]} and standard deviation {statistics.loc[model_best,st]}')
    else:
        model_best = pd.to_numeric(statistics[metric+'_mean']).idxmax()
        print(f'Model with highest {metric} is {model_best} with value {statistics.loc[model_best,mn]} and standard deviation {statistics.loc[model_best,st]}')

In [ ]:
## Print mean and std metrics for each model
for classifier_type in trials_results.classifier.unique():
    print(f'Classifier: {classifier_type}')
    results = trials_results[trials_results['classifier'] == classifier_type]

    # Number of parameters
    parameters = results['Parameters'].values
    print(f'Number of parameters: {parameters[0]}')

    # training time
    trainTime = results['trainTime'].values
    hours, remainder = divmod(trainTime.mean(), 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Mean training time: {hours} hours, {minutes} minutes, and {seconds} seconds, (std {trainTime.std()} sec)')
    print()
    
    # TRAIN results
    metrics = ['BCELoss','Accuracy','Sensitivity','Specificity','ROC_AUC','Precision','F1','WE']
    for metric in metrics:
        values = results['train_' + metric].values
        print(f'Mean train {metric}: {values.mean()}, std {values.std()}')
    print()

    # TEST results
    for metric in metrics:
        values = results['test_' + metric].values
        print(f'Mean test {metric}: {values.mean()}, std {values.std()}')
    print()

    print('-'*120), print()

In [ ]:
## Show boxplots
visualize_boxplots(trials_results,
                   ['test_BCELoss','test_Accuracy','test_F1','test_ROC_AUC','test_WE'], #[c for c in cd_trials_results.columns if 'test_' in c and c not in ['test_TP','test_FP','test_TN','test_FN','test_WE','test_Loss']],
                   True,'Image_classifiers/Boxplots_allModels.png')

In [ ]:
## Statistical model comparison
compare_models(trials_results)

# Model selection

In [ ]:
selected_model = 'multiInputCNN'

In [ ]:
visualize_boxplot_onemodel(trials_results[trials_results['classifier']==selected_model],
                           ['test_Accuracy','test_Sensitivity','test_Specificity','test_F1','test_ROC_AUC'],
                           True,'Image_classifiers/Boxplot_'+selected_model+'.png')

# With segmented frontal images

In [ ]:
import cv2

In [ ]:
## Load segmentation model

path_segmentation = 'Segmentation'

# Select model with highest Dice coefficient
segmentation_results = pd.read_csv(os.path.join(path_segmentation, 'Results.csv'), index_col=0)
best_segmentation_model = segmentation_results.iloc[np.argmax(segmentation_results['testDice'].values)].name
print(best_segmentation_model)

# load the model
segmentation_model = tf.keras.models.load_model(os.path.join(path_segmentation, 'Models', best_segmentation_model+'.h5'), compile = False)

In [ ]:
## Determine threshold to remove background
# Plot the histogram
plt.figure(figsize=(10, 6))
plt.hist(np.concatenate((front, L90, R90)).flatten(), 
         bins=256, range=(0, 1), density=True, color='blue', alpha=0.7)
plt.title('Histogram of Pixel Values')
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.show()

threshold = float(input('Enter threshold value: '))

In [ ]:
## Function to preprocess images: segment the breast region in frontal imagesm + remove background in all views
def apply_segmentation(images, segmentation_model):
    
    ## Segment the breast region in frontal images
    # Resize images to match the segmentation model's input shape
    dsize = segmentation_model.input.shape
    images_reduced = np.asarray([cv2.resize(img, (dsize[2],dsize[1])) for img in images])  # Reduce image size to 240x320
    images_reduced = np.repeat(np.expand_dims(images_reduced,axis=-1), 3, axis=-1)  # Convert to 3-channel image
    
    # Predict masks
    masks = segmentation_model(images_reduced)
    masks = np.round(masks)  # Convert to binary

    # Resize masks back to original image size
    masks = np.asarray([cv2.resize(tf.squeeze(mask).numpy(), (front.shape[2], front.shape[1])) for mask in masks])  # Resize mask to original image shape
    
    images_segmented = np.multiply(images, masks)

    return images_segmented

## Tune learning rate

In [ ]:
## Function to get the data for hyperparameter tuning
def data_segmentation():    
    ## Frontal images
    front_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/Front',f))) for f in os.listdir('Images/Healthy/Front')])
    front_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/Front',f))) for f in os.listdir('Images/Sick/Front')])
    front = np.concatenate((front_images_h, front_images_s))

    ## Left lateral (L90) images
    L90_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/L90',f))) for f in os.listdir('Images/Healthy/L90')])
    L90_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/L90',f))) for f in os.listdir('Images/Sick/L90')])
    L90 = np.concatenate((L90_images_h, L90_images_s))

    ## Right lateral (R90) images
    R90_images_h = np.asarray([np.array(Image.open(os.path.join('Images/Healthy/R90',f))) for f in os.listdir('Images/Healthy/R90')])
    R90_images_s = np.asarray([np.array(Image.open(os.path.join('Images/Sick/R90',f))) for f in os.listdir('Images/Sick/R90')])
    R90 = np.concatenate((R90_images_h, R90_images_s))
    
    ## Labels
    labels_h = [0]*len(front_images_h)
    labels_s = [1]*len(front_images_s)
    labels = np.concatenate((labels_h, labels_s))

    ## Split the dataset into crossval and test sets
    seed_tuning = 13  # Seed for hyperparameter tuning
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=int(round(0.15*len(labels))), random_state = seed_tuning)
    crossval_index, test_index = [[crossval_index, test_index] for crossval_index, test_index in splitter.split(front, labels)][0]
    
    front_crossval = front[crossval_index]
    L90_crossval = L90[crossval_index]
    R90_crossval = R90[crossval_index]
    labels_crossval = labels[crossval_index]

    #front_test = front[test_index]
    #L90_test = L90[test_index]
    #R90_tets = R90[test_index]
    #labels_test = labels[test_index]

    ## Segment frontal images
    # Load segmentation model
    path_segmentation = 'Segmentation'
    segmentation_results = pd.read_csv(os.path.join(path_segmentation, 'Results.csv'), index_col=0)
    best_segmentation_model = segmentation_results.iloc[np.argmax(segmentation_results['testDice'].values)].name
    segmentation_model = tf.keras.models.load_model(os.path.join(path_segmentation, 'Models', best_segmentation_model+'.h5'), compile = False)

    # Apply segmentation
    front_crossval = apply_segmentation(front_crossval, segmentation_model)
    #front_test = apply_segmentation(front_test, segmentation_model)
    
    ## Remove background
    background_threshold = 0.4
    front_crossval = np.multiply(front_crossval, front_crossval > background_threshold)
    L90_crossval = np.multiply(L90_crossval, L90_crossval > background_threshold)
    R90_crossval = np.multiply(R90_crossval, R90_crossval > background_threshold)

    #front_test = np.multiply(front_test, front_test > background_threshold)
    #L90_test = np.multiply(L90_test, L90_test > background_threshold)
    #R90_tets = np.multiply(R90_tets, R90_tets > background_threshold)

    ## Add the channels dimension
    front_crossval = np.expand_dims(front_crossval,-1)
    L90_crossval = np.expand_dims(L90_crossval,-1)
    R90_crossval = np.expand_dims(R90_crossval,-1)

    #front_test = np.expand_dims(front_test,-1)
    #L90_test = np.expand_dims(L90_test,-1)
    #R90_tets = np.expand_dims(R90_tets,-1)

    return front_crossval, L90_crossval, R90_crossval, labels_crossval

In [ ]:
# Obective function to minimize
def objective_multiInput_segmented(params):

    # Load the data
    front, L90, R90, labels = data_segmentation()
    _,h,w,_ = front.shape

    # Initialize cross-validation
    epochs = 30
    batch_size = 8
    lr = params['lr']
    folds = 5
    seed = 13
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

    # Store results from each fold
    fold_metrics = []

    # Initialize the model
    with open("Image_classifiers/Parameters_multiInputCNN.json", "r") as f:
        model_params = json.load(f)
    model_params.pop('lr')
    model_params = {
        key: (tuple(value) if isinstance(value, list) else value)
        for key, value in model_params.items()
    }
    model = create_multiInput_model(model_params, h, w)

    # Save the initial weights of the model
    initial_weights = model.get_weights()

    for i, (train_idx, val_idx) in enumerate(skf.split(front, labels)):
        
        print(f"Fold {i+1}/{folds}")
        
        # Split the data into training and validation sets
        train_front, val_front = front[train_idx], front[val_idx]
        train_L90, val_L90 = L90[train_idx], L90[val_idx]
        train_R90, val_R90 = R90[train_idx], R90[val_idx]
        train_labels, val_labels = labels[train_idx], labels[val_idx]

        rate_train = sum(train_labels==0)/sum(train_labels)

        # Stack views along the channel axis
        train_images = [train_front, train_L90, train_R90]
        val_images = [val_front, val_L90, val_R90]

        # Reset the model to initial weights
        model.set_weights(initial_weights)

        # Train the model
        optimizer = SGD(learning_rate=lr)
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['AUC'])

        history = model.fit(
            train_images, train_labels,
            class_weight={0: 1, 1: rate_train},
            validation_data=(val_images, val_labels),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=[NaNCheckCallback()],
            verbose=0  # Suppress training output
        )

        # Record the best validation AUC for this fold
        best_auc = max(history.history['val_AUC'])
        fold_metrics.append(best_auc)

        print()

    # Clear TensorFlow session to release GPU memory
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    # Return the negative mean AUC across all folds as the loss
    print(f'Max test ROC AUC in each fold: {fold_metrics}')
    mean_auc = np.mean(fold_metrics)
    return -mean_auc  # Maximize AUC by minimizing its negative

In [ ]:
## Hyperparameter search space
space = {
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-2))
}

# Create trials object to store optimization results
trials = Trials()

# Run optimization
ti = time.time()
best_multiInput_segmented = fmin(
    fn=objective_multiInput_segmented,
    space=space,
    algo=tpe.suggest,
    max_evals=max_evals,
    trials=trials
)
tuningTime = time.time() - ti

hours, remainder = divmod(tuningTime, 3600)
minutes, seconds = divmod(remainder, 60)
print(f'Hyperparameter tuning took {hours} hours, {minutes} minutes, and {seconds} seconds.')

lr = best_multiInput_segmented['lr']
print(f'Best learning rate: {lr}')

# Save the tuned learning rate
np.save('Image_classifiers/lr_'+selected_model+'_segmented', lr)

## Train N times

In [ ]:
## Read tuned hyperparameters
# Model architecture
with open("Image_classifiers/Parameters_multiInputCNN.json", "r") as f:
    params = json.load(f)
    
params.pop('lr')

# Convert lists back to tuples if needed
params = {
    key: (tuple(value) if isinstance(value, list) else value)
    for key, value in params.items()
}

# learning rate
lr = np.load('Image_classifiers/lr_'+selected_model+'_segmented')

In [ ]:
trials_results = pd.read_csv('Image_classifiers/Results_'+str(N)+'trials.csv', index_col = 0).to_dict(orient='records')

In [ ]:
splitter = StratifiedShuffleSplit(n_splits=N, test_size=int(round(0.15*len(labels))), random_state = seed)

#best_test_auc = 0
for trial, (train_index, test_index) in enumerate(splitter.split(front, labels)):

    print(f'Trial {trial + 1}'), print()

    ## Split the dataset
    train_front = front[train_index]
    train_L90 = L90[train_index]
    train_R90 = R90[train_index]
    train_labels = labels[train_index]
    
    test_front = front[test_index]
    test_L90 = L90[test_index]
    test_R90 = R90[test_index]
    test_labels = labels[test_index]

    ## Segment frontal images
    train_front = apply_segmentation(train_front, segmentation_model)
    test_front = apply_segmentation(test_front, segmentation_model)
    
    ## Remove background
    train_front = np.multiply(train_front, train_front > background_threshold)
    train_L90 = np.multiply(train_L90, train_L90 > background_threshold)
    train_R90 = np.multiply(train_R90, train_R90 > background_threshold)

    test_front = np.multiply(test_front, test_front > background_threshold)
    test_L90 = np.multiply(test_L90, test_L90 > background_threshold)
    test_R90 = np.multiply(test_R90, test_R90 > background_threshold)

    ## Add the channel dimension
    train_front = np.expand_dims(train_front, -1)
    train_L90 = np.expand_dims(train_L90, -1)
    train_R90 = np.expand_dims(train_R90, -1)

    test_front = np.expand_dims(test_front, -1)
    test_L90 = np.expand_dims(test_L90, -1)
    test_R90 = np.expand_dims(test_R90, -1)

    ## Compute the class weight difference
    rate_train = sum(train_labels == 0) / sum(train_labels)

    ## Initialize and compile the model
    model = create_multiInput_model(params, h, w)
    model.compile(loss='binary_crossentropy',
                  metrics=['accuracy', 'AUC', 'TruePositives', 'FalseNegatives', 'TrueNegatives', 'FalsePositives', 'Precision', 'Recall', Specificity(), WeightedError(rate=20)], 
                  optimizer=SGD(learning_rate=lr, clipvalue=1.0))
    
    ## Train the model
    ti = time.time()
    history = model.fit([train_front, train_L90, train_R90], train_labels, class_weight = {0: 1, 1: rate_train}, 
                        validation_data = ([test_front, test_L90, test_R90], test_labels)
                        batch_size=batch_size, epochs=epochs, verbose=0, callbacks=[NaNCheckCallback()]) 
    trainTime = time.time() - ti

    hours, remainder = divmod(trainTime, 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f'Training took {hours} hours, {minutes} minutes, and {seconds} seconds.')

    # Save learning curves
    plt.figure(figsize=(30, 10))
    plt.subplot(1, 3, 1)
    plt.plot(range(epochs), history.history['loss'], label='Train Loss')
    plt.plot(range(epochs), history.history['val_loss'], label='Test Loss')
    plt.legend(loc='upper right',fontsize=10)
    plt.title('Train and Test Loss',fontsize=12)
    
    plt.subplot(1, 3, 2)
    plt.plot(range(epochs), history.history['accuracy'], label='Training Accuracy')
    plt.plot(range(epochs), history.history['val_accuracy'], label='Validation Accuracy')
    plt.legend(loc='lower right',fontsize=10)
    plt.title('Train and Test Accuracy',fontsize=12)
    
    plt.subplot(1, 3, 3)
    plt.plot(range(epochs), history.history['AUC'], label='Train AUC')
    plt.plot(range(epochs), history.history['val_AUC'], label='Test AUC')
    plt.legend(loc='lower right',fontsize=10)
    plt.title('Train and Test ROC AUC',fontsize=12)
    
    plt.savefig('Image_classifiers/Learning_curves/MultiInputCNN_segmented_'+str(trial+1)+'.png'), plt.show()
    plt.show()

    # Save the model
    model.save('Image_classifiers/Models/MultiInputCNN_segmented_'+str(trial+1)+'.h5')

    # Print the number of parameters in the model
    total_params = model.count_params()  # total_params = np.sum([np.prod(v.shape.as_list()) for v in model.variables])
    trainable_params = np.sum([np.prod(v.shape.as_list()) for v in model.trainable_variables])
    print(f'Classifier has {total_params} total parameters, {trainable_params} of which are trainable.'), print()
    
    # Predict
    train_preds = model.predict([train_front, train_L90, train_R90])
    np.save('Image_classifiers/Predictions/MultiInputCNN_segmented_train_'+str(trial+1)+'.npy',train_preds)
    test_preds = model.predict([test_front, test_L90, test_R90])
    np.save('Image_classifiers/Predictions/MultiInputCNN_segmented_test_'+str(trial+1)+'.npy',test_preds)

    # Evaluate the model
    results_train = evaluate_model_skl(train_preds, train_labels)
    print('TRAIN results:')
    for metric, value in results_train.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    results_test = evaluate_model_skl(test_preds, test_labels)
    print('TEST results:')
    for metric, value in results_test.items():
        print(f'{metric}: {value:.4f}' if isinstance(value, (float, int)) else f'{metric}: {value}')
    print()

    ## Store results
    trials_results.append({**{'classifier':'multiInputCNN_segmented'}, **{'trial':trial+1}, 
                            **store_results(trainable_params, trainTime, results_train, results_test)})

    # Clear TensorFlow session to release GPU memory
    tf.keras.backend.clear_session()
    del model
    gc.collect()

    print(), print(100*'#'), print()

pd.DataFrame(trials_results).to_csv('Image_classifiers/Results_'+str(N)+'trials.csv')